In [11]:
import PyQt6.QtCore
import os
os.environ["QT_API"] = "pyqt5"
import matplotlib.pyplot as plt

%matplotlib qt
import pandas as pd
import seaborn as sns
import numpy as np



In [37]:
# Load the CSV file to check its contents
file_path = 'classification_results_3_fbcsp.csv'
classification_data = pd.read_csv(file_path)

In [33]:
# Load the CSV file to check its contents
file_path = 'classification_results_4_fbcsp_like_muse.csv'
classification_data = pd.read_csv(file_path)

reduced analysis against full

In [39]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the uploaded CSV files
file_1_path = 'classification_results_4_fbcsp_like_muse.csv'
file_3_path = 'classification_results_3_fbcsp.csv'

# Reading the CSV files to explore their structure
data_reduced_channels = pd.read_csv(file_1_path)
data_full_channels_new = pd.read_csv(file_3_path)

# Extracting relevant columns and adding a new column to distinguish between reduced and full channels
data_reduced_channels_plot = data_reduced_channels[['classifier', 'mean_accuracy']].copy()
data_reduced_channels_plot['Channel Type'] = 'Reduced'

data_full_channels_new_plot = data_full_channels_new[['classifier', 'mean_accuracy']].copy()
data_full_channels_new_plot['Channel Type'] = 'Full'

# Combining the reduced channels data with the corrected full channels data for plotting
combined_data_corrected = pd.concat([data_reduced_channels_plot, data_full_channels_new_plot], ignore_index=True)

# Plotting the accuracy for each classifier grouped by channel type
plt.figure(figsize=(12, 6))
for channel_type, group_data in combined_data_corrected.groupby('Channel Type'):
    plt.bar(group_data['classifier'], group_data['mean_accuracy'], alpha=0.6, label=channel_type)

plt.xlabel('Classifier')
plt.ylabel('Mean Accuracy')
plt.title('Classifier Accuracy for Reduced and Full Channel Analysis')
plt.xticks(rotation=45)
plt.legend(title='Channel Type')
plt.tight_layout()
plt.show()


In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load the uploaded CSV files
file_1_path = 'classification_results_4_fbcsp_like_muse.csv'
file_3_path = 'classification_results_3_fbcsp.csv'

# Reading the CSV files
data_reduced_channels = pd.read_csv(file_1_path)
data_full_channels_new = pd.read_csv(file_3_path)

# Extract relevant columns for plotting
data_reduced_channels_plot = data_reduced_channels[['classifier', 'mean_accuracy']].copy()
data_reduced_channels_plot['Channel Type'] = 'Reduced'

data_full_channels_plot = data_full_channels_new[['classifier', 'mean_accuracy']].copy()
data_full_channels_plot['Channel Type'] = 'Full'

# Combine both datasets
combined_data = pd.concat([data_reduced_channels_plot, data_full_channels_plot], ignore_index=True)

# Find the common classifiers in both datasets
common_classifiers = np.intersect1d(data_reduced_channels_plot['classifier'].unique(), data_full_channels_plot['classifier'].unique())

# Filter the data to include only the common classifiers
reduced_data_filtered = data_reduced_channels_plot[data_reduced_channels_plot['classifier'].isin(common_classifiers)]
full_data_filtered = data_full_channels_plot[data_full_channels_plot['classifier'].isin(common_classifiers)]

# Group by classifiers to aggregate mean accuracy
reduced_data_aggregated = reduced_data_filtered.groupby('classifier')['mean_accuracy'].mean().reset_index()
full_data_aggregated = full_data_filtered.groupby('classifier')['mean_accuracy'].mean().reset_index()

# Sort classifiers for consistency in plotting
reduced_data_aggregated.sort_values(by='classifier', inplace=True)
full_data_aggregated.sort_values(by='classifier', inplace=True)

# Extract the sorted list of classifiers
classifiers = reduced_data_aggregated['classifier'].values
positions = np.arange(len(classifiers))
bar_width = 0.35

# Plotting
plt.figure(figsize=(14, 8))

# Plot for reduced channel data
plt.bar(positions - bar_width/2, reduced_data_aggregated['mean_accuracy'], bar_width, label='Reduced', alpha=0.6)

# Plot for full channel data
plt.bar(positions + bar_width/2, full_data_aggregated['mean_accuracy'], bar_width, label='Full', alpha=0.6)

# Customizing the plot
plt.xlabel('Classifier')
plt.ylabel('Mean Accuracy')
plt.title('Classifier Accuracy for Reduced and Full Channel Analysis (Side by Side)')
plt.xticks(positions, classifiers, rotation=45)
plt.legend(title='Channel Type')
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
# Let's revise the code to include error bars using the standard deviation (std_accuracy) for each classifier.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load the uploaded CSV files
file_1_path = 'classification_results_4_fbcsp_like_muse.csv'
file_3_path = 'classification_results_3_fbcsp.csv'

# Reading the CSV files
data_reduced_channels = pd.read_csv(file_1_path)
data_full_channels_new = pd.read_csv(file_3_path)

# Extract relevant columns for plotting
data_reduced_channels_plot = data_reduced_channels[['classifier', 'mean_accuracy', 'std_accuracy']].copy()
data_reduced_channels_plot['Channel Type'] = 'Reduced'

data_full_channels_plot = data_full_channels_new[['classifier', 'mean_accuracy', 'std_accuracy']].copy()
data_full_channels_plot['Channel Type'] = 'Full'

# Combine both datasets
combined_data = pd.concat([data_reduced_channels_plot, data_full_channels_plot], ignore_index=True)

# Find the common classifiers in both datasets
common_classifiers = np.intersect1d(data_reduced_channels_plot['classifier'].unique(), data_full_channels_plot['classifier'].unique())

# Filter the data to include only the common classifiers
reduced_data_filtered = data_reduced_channels_plot[data_reduced_channels_plot['classifier'].isin(common_classifiers)]
full_data_filtered = data_full_channels_plot[data_full_channels_plot['classifier'].isin(common_classifiers)]

# Group by classifiers to aggregate mean accuracy and standard deviation
reduced_data_aggregated = reduced_data_filtered.groupby('classifier').agg({'mean_accuracy': 'mean', 'std_accuracy': 'mean'}).reset_index()
full_data_aggregated = full_data_filtered.groupby('classifier').agg({'mean_accuracy': 'mean', 'std_accuracy': 'mean'}).reset_index()

# Sort classifiers for consistency in plotting
reduced_data_aggregated.sort_values(by='classifier', inplace=True)
full_data_aggregated.sort_values(by='classifier', inplace=True)

# Extract the sorted list of classifiers
classifiers = reduced_data_aggregated['classifier'].values
positions = np.arange(len(classifiers))
bar_width = 0.35

# Plotting
plt.figure(figsize=(14, 8))

# Plot for reduced channel data with error bars
plt.bar(positions - bar_width/2, reduced_data_aggregated['mean_accuracy'], bar_width, 
        yerr=reduced_data_aggregated['std_accuracy'], capsize=5, label='Reduced', alpha=0.6)

# Plot for full channel data with error bars
plt.bar(positions + bar_width/2, full_data_aggregated['mean_accuracy'], bar_width, 
        yerr=full_data_aggregated['std_accuracy'], capsize=5, label='Full', alpha=0.6)

# Customizing the plot
plt.xlabel('Classifier')
plt.ylabel('Mean Accuracy')
plt.title('Classifier Accuracy for Reduced and Full Channel Analysis (With Error Bars)')
plt.xticks(positions, classifiers, rotation=45)
plt.legend(title='Channel Type')
plt.tight_layout()

# Show the plot
plt.show()



In [45]:
reduced_data_aggregated

,classifier,mean_accuracy
0,KNN,0.677723
1,LDA,0.693343
2,LogisticRegression,0.692464
3,RandomForest,0.690748
4,SVM,0.709098


In [44]:
full_data_aggregated

,classifier,mean_accuracy
0,KNN,0.943317
1,LDA,0.934926
2,LogisticRegression,0.939114
3,RandomForest,0.946484
4,SVM,0.948708


In [38]:
import matplotlib.pyplot as plt

# Group by classifier and calculate the mean and standard deviation
classifier_summary = classification_data.groupby('classifier').agg(
    mean_accuracy=('mean_accuracy', 'mean'),
    std_accuracy=('std_accuracy', 'mean')
).reset_index()

# Create the plot
plt.figure(figsize=(10,6))
plt.bar(classifier_summary['classifier'], classifier_summary['mean_accuracy'], 
        yerr=classifier_summary['std_accuracy'], capsize=5)

# Customize the plot
plt.xlabel('Classifier')
plt.ylabel('Average Accuracy')
plt.title('Average Classification Accuracy by Classifier (with Error Bars)')
plt.xticks(rotation=45)
plt.tight_layout()

# Show the plot
plt.show()


In [14]:
new_file_path = 'classification_results_3FBCSP_perband_summary.csv'
new_classification_data = pd.read_csv(new_file_path)

In [40]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

classifiers = new_classification_data['classifier'].unique()
bar_width = 0.15
positions = np.arange(len(new_classification_data['band'].unique()))

# Reorder the bands to be 'theta', 'alpha', 'beta'
new_classification_data['band'] = pd.Categorical(new_classification_data['band'], categories=['theta', 'alpha', 'beta'], ordered=True)

# Sort the data by the new band order
new_classification_data = new_classification_data.sort_values('band')


# Replot with the reordered bands
plt.figure(figsize=(12,8))

# Loop through classifiers and plot each one separately with reordered bands
for i, classifier in enumerate(classifiers):
    classifier_data = new_classification_data[new_classification_data['classifier'] == classifier]
    offset_positions = [pos + i * bar_width for pos in positions]
    
    plt.bar(offset_positions, classifier_data['avg_accuracy'], bar_width, 
            yerr=classifier_data['std_accuracy'], capsize=5, label=classifier)

# Customize the plot
plt.xlabel('Frequency Band')
plt.ylabel('Average Accuracy')
plt.title('Average Classification Accuracy by Classifier and Frequency Band (with Error Bars)')
plt.xticks([pos + bar_width * (len(classifiers) / 2 - 0.5) for pos in positions], ['theta', 'alpha', 'beta'])  # Center the ticks
plt.legend(title='Classifier')
plt.tight_layout()

# Show the plot
plt.show()


In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Load the data
logo_detailed_file_path = 'Powerband_classification_results_perband.csv'
logo_detailed_data = pd.read_csv(logo_detailed_file_path)

# Set the classifiers and other plotting parameters
classifiers = logo_detailed_data['classifier'].unique()
bar_width = 0.15
positions = np.arange(len(logo_detailed_data['band'].unique()))

# Reorder the bands to be 'theta', 'alpha', 'beta'
logo_detailed_data['band'] = pd.Categorical(logo_detailed_data['band'], categories=['theta', 'alpha', 'beta'], ordered=True)

# Sort the data by the new band order
logo_detailed_data = logo_detailed_data.sort_values('band')

# Replot with the reordered bands
plt.figure(figsize=(12, 8))

# Loop through classifiers and plot each one separately with reordered bands
for i, classifier in enumerate(classifiers):
    classifier_data = logo_detailed_data[logo_detailed_data['classifier'] == classifier]
    
    # Aggregate mean and std accuracy for each band
    aggregated_data = classifier_data.groupby('band').agg(
        mean_accuracy=('mean_accuracy', 'mean'),
        std_accuracy=('std_accuracy', 'mean')  # Taking mean of stds for simplicity
    ).loc[['theta', 'alpha', 'beta']]
    
    offset_positions = [pos + i * bar_width for pos in positions]
    
    plt.bar(offset_positions, aggregated_data['mean_accuracy'], bar_width, 
            yerr=aggregated_data['std_accuracy'], capsize=5, label=classifier)

# Customize the plot
plt.xlabel('Frequency Band')
plt.ylabel('Average Accuracy')
plt.title('Average Classification Accuracy by Classifier and Frequency Band (with Error Bars)')
plt.xticks([pos + bar_width * (len(classifiers) / 2 - 0.5) for pos in positions], ['theta', 'alpha', 'beta'])  # Center the ticks
plt.legend(title='Classifier')
plt.tight_layout()

# Show the plot
plt.show()


In [20]:
import pandas as pd
import numpy as np
from scipy.stats import shapiro, friedmanchisquare, wilcoxon
from statsmodels.stats.anova import AnovaRM
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multitest import multipletests
from itertools import combinations
from scipy.stats import ttest_rel

# Load the data
data = pd.read_csv('Powerband_classification_results_perband.csv')

# List of classifiers
classifiers = data['classifier'].unique()

# Initialize a results dictionary
results = {}

# For each classifier
for clf in classifiers:
    clf_data = data[data['classifier'] == clf]
    bands = clf_data['band'].unique()
    accuracies = {}
    
    # Collect accuracies for each band
    for band in bands:
        band_data = clf_data[clf_data['band'] == band]
        accuracies[band] = band_data['mean_accuracy'].values
    
    # Check for normality
    normality_pvalues = {}
    for band in bands:
        stat, p = shapiro(accuracies[band])
        normality_pvalues[band] = p
    
    # Decide whether data is normally distributed
    normal = all(p > 0.05 for p in normality_pvalues.values())
    
    # Initialize lists to store pairwise comparison results
    p_values = []
    comparisons = []
    effect_sizes = []
    
    # Pairwise combinations of bands
    band_pairs = list(combinations(bands, 2))
    
    if normal:
        # Perform repeated measures ANOVA
        anova_data = pd.DataFrame({
            'participant': np.tile(range(1, len(accuracies[bands[0]]) + 1), len(bands)),
            'band': np.repeat(bands, len(accuracies[bands[0]])),
            'accuracy': np.concatenate([accuracies[band] for band in bands])
        })
        anova = AnovaRM(anova_data, 'accuracy', 'participant', within=['band'])
        anova_results = anova.fit()
        print(f'\nClassifier: {clf}')
        print(anova_results)
        
        # Perform pairwise t-tests with Bonferroni correction
        for (band1, band2) in band_pairs:
            stat, p = ttest_rel(accuracies[band1], accuracies[band2])
            p_values.append(p)
            comparisons.append((band1, band2))
            # Calculate Cohen's d for effect size
            diff = accuracies[band1] - accuracies[band2]
            d = diff.mean() / diff.std(ddof=1)
            effect_sizes.append(d)
    else:
        # Perform Friedman test
        data_matrix = np.array([accuracies[band] for band in bands]).T
        stat, p = friedmanchisquare(*[data_matrix[:, i] for i in range(data_matrix.shape[1])])
        print(f'\nClassifier: {clf}')
        print(f'Friedman test statistic={stat:.4f}, p-value={p:.4f}')
        
        # Perform pairwise Wilcoxon tests with Bonferroni correction
        for (band1, band2) in band_pairs:
            stat, p = wilcoxon(accuracies[band1], accuracies[band2])
            p_values.append(p)
            comparisons.append((band1, band2))
            # Calculate effect size r
            z = (stat - (len(accuracies[band1]) * (len(accuracies[band1]) + 1) / 4)) / np.sqrt(len(accuracies[band1]) * (len(accuracies[band1]) + 1) * (2 * len(accuracies[band1]) + 1) / 24)
            r = z / np.sqrt(len(accuracies[band1]))
            effect_sizes.append(r)
    
    # Apply Bonferroni correction
    corrected_p = multipletests(p_values, method='bonferroni')[1]
    
    # Store results
    results[clf] = {
        'comparisons': comparisons,
        'p_values': p_values,
        'corrected_p': corrected_p,
        'effect_sizes': effect_sizes
    }
    
    # Print pairwise comparison results
    for i in range(len(comparisons)):
        comp = comparisons[i]
        print(f'Comparison {comp}: p-value={p_values[i]:.4f}, corrected p-value={corrected_p[i]:.4f}, effect size={effect_sizes[i]:.4f}')



Classifier: LogisticRegression
Friedman test statistic=39.0000, p-value=0.0000
Comparison ('theta', 'alpha'): p-value=0.0043, corrected p-value=0.0130, effect size=-0.5657
Comparison ('theta', 'beta'): p-value=0.0000, corrected p-value=0.0000, effect size=-0.8748
Comparison ('alpha', 'beta'): p-value=0.0000, corrected p-value=0.0000, effect size=-0.8748

Classifier: SVM
Friedman test statistic=36.0833, p-value=0.0000
Comparison ('theta', 'alpha'): p-value=0.8996, corrected p-value=1.0000, effect size=-0.0292
Comparison ('theta', 'beta'): p-value=0.0000, corrected p-value=0.0000, effect size=-0.8748
Comparison ('alpha', 'beta'): p-value=0.0000, corrected p-value=0.0000, effect size=-0.8748

Classifier: LDA
Friedman test statistic=41.3333, p-value=0.0000
Comparison ('theta', 'alpha'): p-value=0.0010, corrected p-value=0.0029, effect size=-0.6415
Comparison ('theta', 'beta'): p-value=0.0000, corrected p-value=0.0000, effect size=-0.8748
Comparison ('alpha', 'beta'): p-value=0.0000, corre

In [30]:
import pandas as pd
import numpy as np
from scipy.stats import shapiro, friedmanchisquare, wilcoxon
from statsmodels.stats.anova import AnovaRM
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multitest import multipletests
from itertools import combinations
from scipy.stats import ttest_rel

# Load the data
data = pd.read_csv('classification_results_3FBCSP__perband_detailed.csv')

# List of classifiers
classifiers = data['classifier'].unique()

# Initialize a results dictionary
results = {}

# For each classifier
for clf in classifiers:
    clf_data = data[data['classifier'] == clf]
    bands = clf_data['band'].unique()
    accuracies = {}
    
    # Collect accuracies for each band
    for band in bands:
        band_data = clf_data[clf_data['band'] == band]
        accuracies[band] = band_data['mean_accuracy'].values
    
    # Check for normality
    normality_pvalues = {}
    for band in bands:
        stat, p = shapiro(accuracies[band])
        normality_pvalues[band] = p
    
    # Decide whether data is normally distributed
    normal = all(p > 0.05 for p in normality_pvalues.values())
    
    # Initialize lists to store pairwise comparison results
    p_values = []
    comparisons = []
    effect_sizes = []
    
    # Pairwise combinations of bands
    band_pairs = list(combinations(bands, 2))
    
    if normal:
        # Perform repeated measures ANOVA
        anova_data = pd.DataFrame({
            'participant': np.tile(range(1, len(accuracies[bands[0]]) + 1), len(bands)),
            'band': np.repeat(bands, len(accuracies[bands[0]])),
            'accuracy': np.concatenate([accuracies[band] for band in bands])
        })
        anova = AnovaRM(anova_data, 'accuracy', 'participant', within=['band'])
        anova_results = anova.fit()
        print(f'\nClassifier: {clf}')
        print(anova_results)
        
        # Perform pairwise t-tests with Bonferroni correction
        for (band1, band2) in band_pairs:
            stat, p = ttest_rel(accuracies[band1], accuracies[band2])
            p_values.append(p)
            comparisons.append((band1, band2))
            # Calculate Cohen's d for effect size
            diff = accuracies[band1] - accuracies[band2]
            d = diff.mean() / diff.std(ddof=1)
            effect_sizes.append(d)
    else:
        # Perform Friedman test
        data_matrix = np.array([accuracies[band] for band in bands]).T
        stat, p = friedmanchisquare(*[data_matrix[:, i] for i in range(data_matrix.shape[1])])
        print(f'\nClassifier: {clf}')
        print(f'Friedman test statistic={stat:.4f}, p-value={p:.4f}')
        
        # Perform pairwise Wilcoxon tests with Bonferroni correction
        for (band1, band2) in band_pairs:
            stat, p = wilcoxon(accuracies[band1], accuracies[band2])
            p_values.append(p)
            comparisons.append((band1, band2))
            # Calculate effect size r
            z = (stat - (len(accuracies[band1]) * (len(accuracies[band1]) + 1) / 4)) / np.sqrt(len(accuracies[band1]) * (len(accuracies[band1]) + 1) * (2 * len(accuracies[band1]) + 1) / 24)
            r = z / np.sqrt(len(accuracies[band1]))
            effect_sizes.append(r)
    
    # Apply Bonferroni correction
    corrected_p = multipletests(p_values, method='bonferroni')[1]
    
    # Store results
    results[clf] = {
        'comparisons': comparisons,
        'p_values': p_values,
        'corrected_p': corrected_p,
        'effect_sizes': effect_sizes
    }
    
    # Print pairwise comparison results
    for i in range(len(comparisons)):
        comp = comparisons[i]
        print(f'Comparison {comp}: p-value={p_values[i]:.4f}, corrected p-value={corrected_p[i]:.4f}, effect size={effect_sizes[i]:.4f}')



Classifier: LogisticRegression
Friedman test statistic=22.7500, p-value=0.0000
Comparison ('theta', 'alpha'): p-value=0.0011, corrected p-value=0.0033, effect size=-0.6357
Comparison ('theta', 'beta'): p-value=0.0000, corrected p-value=0.0001, effect size=-0.7582
Comparison ('alpha', 'beta'): p-value=0.0004, corrected p-value=0.0011, effect size=-0.6824

Classifier: SVM
Friedman test statistic=33.5833, p-value=0.0000
Comparison ('theta', 'alpha'): p-value=0.0005, corrected p-value=0.0015, effect size=-0.6707
Comparison ('theta', 'beta'): p-value=0.0000, corrected p-value=0.0000, effect size=-0.8573
Comparison ('alpha', 'beta'): p-value=0.0000, corrected p-value=0.0000, effect size=-0.8048

Classifier: LDA
              Anova
     F Value Num DF  Den DF Pr > F
----------------------------------
band 19.1178 2.0000 46.0000 0.0000

Comparison ('theta', 'alpha'): p-value=0.0010, corrected p-value=0.0031, effect size=-0.7670
Comparison ('theta', 'beta'): p-value=0.0001, corrected p-value=0

In [8]:
logo_detailed_file_path = 'classification_results_3FBCSP_logo_detailed.csv'
logo_detailed_data = pd.read_csv(logo_detailed_file_path)
logo_detailed_data.head()

,classifier,participant_left_out,score
0,LogisticRegression,1,0.537998
1,LogisticRegression,2,0.399763
2,LogisticRegression,3,0.504478
3,LogisticRegression,4,0.519789
4,LogisticRegression,6,0.495364


In [9]:
# Calculate the standard deviation of accuracy for each classifier
accuracy_stats_per_classifier = logo_detailed_data.groupby('classifier')['score'].agg(['mean', 'std']).reset_index()

# Create a bar plot with error bars representing the standard deviation
plt.figure(figsize=(10,6))
plt.bar(accuracy_stats_per_classifier['classifier'], accuracy_stats_per_classifier['mean'], 
        yerr=accuracy_stats_per_classifier['std'], capsize=5)

# Customize the plot
plt.xlabel('Classifier')
plt.ylabel('Average Accuracy')
plt.title('Average Classification Accuracy by Classifier (with Error Bars)')
plt.xticks(rotation=45)
plt.tight_layout()

# Show the plot
plt.show()


NameError: name 'plt' is not defined

In [7]:
import pandas as pd
# Load all three CSV files to check their contents
file1_path = 'classification_results.csv'
file2_path = 'classification_results_2.csv'
file3_path = 'classification_results_3_fbcsp.csv'

# Load the data
classification1_data = pd.read_csv(file1_path)
classification2_data = pd.read_csv(file2_path)
classification3_data = pd.read_csv(file3_path)

# Display the first few rows of each to understand their structure
classification1_data.head(), classification2_data.head(), classification3_data.head()


(   participant          classifier  mean_accuracy  std_accuracy
 0            1  LogisticRegression       0.702001      0.027968
 1            1                 LDA       0.692377      0.027743
 2            1                 SVM       0.668258      0.044061
 3            1        RandomForest       0.624834      0.028296
 4            1                 KNN       0.569361      0.025416,
    participant          classifier  mean_accuracy  std_accuracy
 0            1  LogisticRegression       0.815400      0.039922
 1            1                 SVM       0.846776      0.021968
 2            1                 LDA       0.799723      0.040773
 3            1                 KNN       0.821475      0.018075
 4            1        RandomForest       0.844359      0.019117,
    participant          classifier  mean_accuracy  std_accuracy
 0            1  LogisticRegression       0.819051      0.036782
 1            1                 SVM       0.848003      0.025710
 2            1        

In [10]:
# Add a column to each dataset to indicate the feature extraction method
classification1_data['feature_extraction'] = 'Powerband features'
classification2_data['feature_extraction'] = 'CSP filters'
classification3_data['feature_extraction'] = 'Filterbank CSP filters'

# Combine all datasets into one
combined_data = pd.concat([classification1_data, classification2_data, classification3_data])

# Create a boxplot to show classifier performance grouped by feature extraction method
plt.figure(figsize=(12, 6))
sns.boxplot(x='feature_extraction', y='mean_accuracy', hue='classifier', data=combined_data, notch = True)

# Customize the plot
plt.xlabel('Feature Extraction Method')
plt.ylabel('Classification Accuracy')
plt.title('Classifier Performance by Feature Extraction Method')
plt.xticks(rotation=45)
plt.tight_layout()

# Show the plot
plt.show()


NameError: name 'plt' is not defined

In [8]:
# Extract accuracy values for each classifier grouped by participants for statistical analysis
# Include all participant accuracies for each feature extraction method
from scipy.stats import shapiro, f_oneway, wilcoxon

# Prepare data for each classifier
statistical_results = {}
classifiers = classification1_data['classifier'].unique()

valid_pids = [el for el in range(1, 32) if el not in [5, 13, 14, 16, 17, 20, 31]]

# Filter each dataset to include only the valid participants
classification1_data = classification1_data[classification1_data['participant'].isin(valid_pids)]
classification2_data = classification2_data[classification2_data['participant'].isin(valid_pids)]
classification3_data = classification3_data[classification3_data['participant'].isin(valid_pids)]

print("1 len: " + str(len(classification1_data)))
print("2 len: " + str(len(classification2_data)))
print("3 len: " + str(len(classification3_data)))


# Perform normality test for Logistic Regression across all feature extraction methods
classification1_data_temp = classification1_data[classification1_data['classifier'] == 'LogisticRegression']['mean_accuracy']
classification2_data_temp = classification2_data[classification2_data['classifier'] == 'LogisticRegression']['mean_accuracy']
classification3_data_temp = classification3_data[classification3_data['classifier'] == 'LogisticRegression']['mean_accuracy']

# Combine all accuracies for Logistic Regression
logreg_all_accuracies = pd.concat([classification1_data_temp, classification2_data_temp, classification3_data_temp])

# Normality test using Shapiro-Wilk for Logistic Regression
normality_test_stat, normality_p_value = shapiro(logreg_all_accuracies)

# Check if the data is normally distributed
if normality_p_value > 0.05:
    # Data is normally distributed; use one-way ANOVA (F-test) for comparisons
    for classifier in classifiers:
        # Get participant accuracies for the current classifier across feature extraction methods
        acc_powerband = classification1_data[classification1_data['classifier'] == classifier]['mean_accuracy']
        acc_csp = classification2_data[classification2_data['classifier'] == classifier]['mean_accuracy']
        acc_fbcsp = classification3_data[classification3_data['classifier'] == classifier]['mean_accuracy']

        # Perform one-way ANOVA test
        f_stat, p_value = f_oneway(acc_powerband, acc_csp, acc_fbcsp)
        statistical_results[classifier] = ('ANOVA', f_stat, p_value)
else:
    # Data is not normally distributed; use Wilcoxon rank-sum test for pairwise comparisons
    for classifier in classifiers:
        pairwise_results = {}
        
        # Get participant accuracies for each pair of feature extraction methods
        acc_powerband = classification1_data[classification1_data['classifier'] == classifier]['mean_accuracy']
        acc_csp = classification2_data[classification2_data['classifier'] == classifier]['mean_accuracy']
        acc_fbcsp = classification3_data[classification3_data['classifier'] == classifier]['mean_accuracy']

        # Powerband vs CSP
        wilcoxon_stat_1, p_value_1 = wilcoxon(acc_powerband, acc_csp)
        pairwise_results['Powerband vs CSP'] = (wilcoxon_stat_1, p_value_1)
        
        # Powerband vs Filterbank CSP
        wilcoxon_stat_2, p_value_2 = wilcoxon(acc_powerband, acc_fbcsp)
        pairwise_results['Powerband vs Filterbank CSP'] = (wilcoxon_stat_2, p_value_2)
        
        # CSP vs Filterbank CSP
        wilcoxon_stat_3, p_value_3 = wilcoxon(acc_csp, acc_fbcsp)
        pairwise_results['CSP vs Filterbank CSP'] = (wilcoxon_stat_3, p_value_3)

        statistical_results[classifier] = ('Wilcoxon', pairwise_results)

# Display the results
statistical_results


1 len: 120
2 len: 120
3 len: 120


{'LogisticRegression': ('Wilcoxon',
  {'Powerband vs CSP': (np.float64(39.0), np.float64(0.0008462667465209961)),
   'Powerband vs Filterbank CSP': (np.float64(4.0),
    np.float64(8.344650268554688e-07)),
   'CSP vs Filterbank CSP': (np.float64(19.0),
    np.float64(3.6597251892089844e-05))}),
 'LDA': ('Wilcoxon',
  {'Powerband vs CSP': (np.float64(15.0), np.float64(1.633167266845703e-05)),
   'Powerband vs Filterbank CSP': (np.float64(1.0),
    np.float64(2.384185791015625e-07)),
   'CSP vs Filterbank CSP': (np.float64(10.0),
    np.float64(5.125999450683594e-06))}),
 'SVM': ('Wilcoxon',
  {'Powerband vs CSP': (np.float64(6.0), np.float64(1.6689300537109375e-06)),
   'Powerband vs Filterbank CSP': (np.float64(1.0),
    np.float64(2.384185791015625e-07)),
   'CSP vs Filterbank CSP': (np.float64(30.0),
    np.float64(0.00023925304412841797))}),
 'RandomForest': ('Wilcoxon',
  {'Powerband vs CSP': (np.float64(4.0), np.float64(8.344650268554688e-07)),
   'Powerband vs Filterbank CSP': (n

In [16]:
# Define a dictionary to store the mean accuracies
mean_accuracies = []

# Iterate through each classifier to calculate the mean accuracy for each feature extraction method
for classifier in classifiers:
    # Calculate mean accuracies for the current classifier
    mean_acc_powerband = classification1_data[classification1_data['classifier'] == classifier]['mean_accuracy'].mean()
    mean_acc_csp = classification2_data[classification2_data['classifier'] == classifier]['mean_accuracy'].mean()
    mean_acc_fbcsp = classification3_data[classification3_data['classifier'] == classifier]['mean_accuracy'].mean()
    
    # Store results in a list of dictionaries
    mean_accuracies.append({
        'Classifier': classifier,
        'Powerband Mean Accuracy': mean_acc_powerband,
        'CSP Mean Accuracy': mean_acc_csp,
        'Filterbank CSP Mean Accuracy': mean_acc_fbcsp
    })

# Convert to a DataFrame for easier viewing
mean_accuracies_df = pd.DataFrame(mean_accuracies)
mean_accuracies

[{'Classifier': 'LogisticRegression',
  'Powerband Mean Accuracy': np.float64(0.8522539273333334),
  'CSP Mean Accuracy': np.float64(0.913709085875),
  'Filterbank CSP Mean Accuracy': np.float64(0.9391142743237282)},
 {'Classifier': 'LDA',
  'Powerband Mean Accuracy': np.float64(0.8253539350833333),
  'CSP Mean Accuracy': np.float64(0.9063081445416667),
  'Filterbank CSP Mean Accuracy': np.float64(0.9349256651099327)},
 {'Classifier': 'SVM',
  'Powerband Mean Accuracy': np.float64(0.8377550545416668),
  'CSP Mean Accuracy': np.float64(0.9343010077083335),
  'Filterbank CSP Mean Accuracy': np.float64(0.9487076020602453)},
 {'Classifier': 'RandomForest',
  'Powerband Mean Accuracy': np.float64(0.8117575702500001),
  'CSP Mean Accuracy': np.float64(0.9319298104166668),
  'Filterbank CSP Mean Accuracy': np.float64(0.9464839137053271)},
 {'Classifier': 'KNN',
  'Powerband Mean Accuracy': np.float64(0.7543674976250001),
  'CSP Mean Accuracy': np.float64(0.9313536300833333),
  'Filterbank CSP

In [10]:
from statsmodels.stats.multitest import multipletests

# Extract all p-values from the statistical results for correction
p_values = []
comparisons = []

for classifier, (test_type, results) in statistical_results.items():
    if isinstance(results, dict):  # Ensure there are pairwise comparisons
        for comparison, (stat, p_value) in results.items():
            p_values.append(p_value)
            comparisons.append((classifier, comparison))

# Apply Bonferroni and Benjamini-Hochberg corrections
bonferroni_corrected = multipletests(p_values, method='bonferroni')
bh_corrected = multipletests(p_values, method='fdr_bh')

# Collect and format the results
correction_results = []

for i, (classifier, comparison) in enumerate(comparisons):
    original_p = p_values[i]
    bonf_corrected_p = bonferroni_corrected[1][i]
    bh_corrected_p = bh_corrected[1][i]
    correction_results.append({
        'Classifier': classifier,
        'Comparison': comparison,
        'Original p-value': original_p,
        'Bonferroni-corrected p-value': bonf_corrected_p,
        'Benjamini-Hochberg-corrected p-value': bh_corrected_p
    })

# Display the corrected p-values
correction_results_df = pd.DataFrame(correction_results)

correction_results_df


,Classifier,Comparison,Original p-value,Bonferroni-corrected p-value,Benjamini-Hochberg-corrected p-value
0,LogisticRegression,Powerband vs CSP,8.462667e-04,0.012694,9.067144e-04
1,LogisticRegression,Powerband vs Filterbank CSP,8.344650e-07,0.000013,1.788139e-06
2,LogisticRegression,CSP vs Filterbank CSP,3.659725e-05,0.000549,4.990534e-05
3,LDA,Powerband vs CSP,1.633167e-05,0.000245,2.449751e-05
4,LDA,Powerband vs Filterbank CSP,2.384186e-07,0.000004,8.940697e-07
5,LDA,CSP vs Filterbank CSP,5.125999e-06,0.000077,8.543332e-06
6,SVM,Powerband vs CSP,1.668930e-06,0.000025,3.129244e-06
7,SVM,Powerband vs Filterbank CSP,2.384186e-07,0.000004,8.940697e-07
8,SVM,CSP vs Filterbank CSP,2.392530e-04,0.003589,2.760612e-04
9,RandomForest,Powerband vs CSP,8.344650e-07,0.000013,1.788139e-06


In [11]:
# Function to calculate rank-biserial correlation for Wilcoxon signed-rank test
def rank_biserial(w_stat, n):
    return 1 - (2 * w_stat / (n * (n + 1) / 2))

# Calculate effect sizes (rank-biserial correlation) for each pairwise Wilcoxon test
effect_sizes = []

for classifier, (test_type, results) in statistical_results.items():
    if isinstance(results, dict):  # Ensure there are pairwise comparisons
        for comparison, (w_stat, p_value) in results.items():
            # Number of observations (valid PIDs)
            n = len(valid_pids)

            # Calculate rank-biserial correlation
            rb_correlation = rank_biserial(w_stat, n)

            # Store the results
            effect_sizes.append({
                'Classifier': classifier,
                'Comparison': comparison,
                'Wilcoxon Stat': w_stat,
                'p-value': p_value,
                'Rank-Biserial Correlation': rb_correlation
            })

# Create a DataFrame for the effect sizes
effect_sizes_df = pd.DataFrame(effect_sizes)

effect_sizes_df


,Classifier,Comparison,Wilcoxon Stat,p-value,Rank-Biserial Correlation
0,LogisticRegression,Powerband vs CSP,39.0,8.462667e-04,0.740000
1,LogisticRegression,Powerband vs Filterbank CSP,4.0,8.344650e-07,0.973333
2,LogisticRegression,CSP vs Filterbank CSP,19.0,3.659725e-05,0.873333
3,LDA,Powerband vs CSP,15.0,1.633167e-05,0.900000
4,LDA,Powerband vs Filterbank CSP,1.0,2.384186e-07,0.993333
5,LDA,CSP vs Filterbank CSP,10.0,5.125999e-06,0.933333
6,SVM,Powerband vs CSP,6.0,1.668930e-06,0.960000
7,SVM,Powerband vs Filterbank CSP,1.0,2.384186e-07,0.993333
8,SVM,CSP vs Filterbank CSP,30.0,2.392530e-04,0.800000
9,RandomForest,Powerband vs CSP,4.0,8.344650e-07,0.973333


In [23]:
print(classification1_data.columns)

AttributeError: 'Series' object has no attribute 'columns'

In [19]:
# Perform normality test for Logistic Regression across all feature extraction methods
logreg_accuracies_powerband = classification1_data[classification1_data['classifier'] == 'LogisticRegression']['mean_accuracy']
logreg_accuracies_csp = classification2_data[classification2_data['classifier'] == 'LogisticRegression']['mean_accuracy']
logreg_accuracies_fbcsp = classification3_data[classification3_data['classifier'] == 'LogisticRegression']['mean_accuracy']

# Combine all accuracies for Logistic Regression
logreg_all_accuracies = pd.concat([logreg_accuracies_powerband, logreg_accuracies_csp, logreg_accuracies_fbcsp])

# Normality test using Shapiro-Wilk for Logistic Regression
normality_test_stat, normality_p_value = shapiro(logreg_all_accuracies)

# Prepare data for each classifier
statistical_results = {}
classifiers = classification1_data['classifier'].unique()

# Check if the data is normally distributed
if normality_p_value > 0.05:
    # Data is normally distributed; use one-way ANOVA (F-test) for comparisons
    for classifier in classifiers:
        # Get participant accuracies for the current classifier across feature extraction methods
        acc_powerband = classification1_data[classification1_data['classifier'] == classifier]['mean_accuracy']
        acc_csp = classification2_data[classification2_data['classifier'] == classifier]['mean_accuracy']
        acc_fbcsp = classification3_data[classification3_data['classifier'] == classifier]['mean_accuracy']

        # Perform one-way ANOVA test
        f_stat, p_value = f_oneway(acc_powerband, acc_csp, acc_fbcsp)
        statistical_results[classifier] = ('ANOVA', f_stat, p_value)
else:
    # Data is not normally distributed; use Wilcoxon rank-sum test for pairwise comparisons
    for classifier in classifiers:
        pairwise_results = {}

        # Get participant accuracies for each pair of feature extraction methods
        acc_powerband = classification1_data[classification1_data['classifier'] == classifier]['mean_accuracy']
        acc_csp = classification2_data[classification2_data['classifier'] == classifier]['mean_accuracy']
        acc_fbcsp = classification3_data[classification3_data['classifier'] == classifier]['mean_accuracy']

        # Powerband vs CSP
        wilcoxon_stat_1, p_value_1 = wilcoxon(acc_powerband, acc_csp)
        pairwise_results['Powerband vs CSP'] = (wilcoxon_stat_1, p_value_1)

        # Powerband vs Filterbank CSP
        wilcoxon_stat_2, p_value_2 = wilcoxon(acc_powerband, acc_fbcsp)
        pairwise_results['Powerband vs Filterbank CSP'] = (wilcoxon_stat_2, p_value_2)

        # CSP vs Filterbank CSP
        wilcoxon_stat_3, p_value_3 = wilcoxon(acc_csp, acc_fbcsp)
        pairwise_results['CSP vs Filterbank CSP'] = (wilcoxon_stat_3, p_value_3)

        statistical_results[classifier] = ('Wilcoxon', pairwise_results)

# Display the results
statistical_results


{'LogisticRegression': ('Wilcoxon',
  {'Powerband vs CSP': (np.float64(39.0), np.float64(0.0008462667465209961)),
   'Powerband vs Filterbank CSP': (np.float64(4.0),
    np.float64(8.344650268554688e-07)),
   'CSP vs Filterbank CSP': (np.float64(19.0),
    np.float64(3.6597251892089844e-05))}),
 'LDA': ('Wilcoxon',
  {'Powerband vs CSP': (np.float64(15.0), np.float64(1.633167266845703e-05)),
   'Powerband vs Filterbank CSP': (np.float64(1.0),
    np.float64(2.384185791015625e-07)),
   'CSP vs Filterbank CSP': (np.float64(10.0),
    np.float64(5.125999450683594e-06))}),
 'SVM': ('Wilcoxon',
  {'Powerband vs CSP': (np.float64(6.0), np.float64(1.6689300537109375e-06)),
   'Powerband vs Filterbank CSP': (np.float64(1.0),
    np.float64(2.384185791015625e-07)),
   'CSP vs Filterbank CSP': (np.float64(30.0),
    np.float64(0.00023925304412841797))}),
 'RandomForest': ('Wilcoxon',
  {'Powerband vs CSP': (np.float64(4.0), np.float64(8.344650268554688e-07)),
   'Powerband vs Filterbank CSP': (n

In [28]:
from statsmodels.stats.multitest import multipletests

# Extract all p-values from the statistical results for correction
p_values = []
comparisons = []
rb_correlations = []

def rank_biserial(w_stat, n):
    return 1 - (2 * w_stat / (n * (n + 1) / 2))

for classifier, (test_type, results) in statistical_results.items():
    if isinstance(results, dict):  # Ensure there are pairwise comparisons
        for comparison, (stat, p_value) in results.items():
            n = len(valid_pids)

            # Calculate rank-biserial correlation
            rb_correlation = rank_biserial(stat, n)
            rb_correlations.append(rb_correlation)

            p_values.append(p_value)
            comparisons.append((classifier, comparison))

# Apply Bonferroni and Benjamini-Hochberg corrections
bonferroni_corrected = multipletests(p_values, method='bonferroni')
bh_corrected = multipletests(p_values, method='fdr_bh')

# Collect and format the results
correction_results = []

for i, (classifier, comparison) in enumerate(comparisons):
    original_p = p_values[i]
    bonf_corrected_p = bonferroni_corrected[1][i]
    bh_corrected_p = bh_corrected[1][i]
    correction_results.append({
        'Classifier': classifier,
        'Comparison': comparison,
        # 'Original p-value': original_p,
        'Bonferroni-corrected p-value': bonf_corrected_p,
        # 'Benjamini-Hochberg-corrected p-value': bh_corrected_p,
         'Rank-Biserial Correlation': rb_correlations[i]
    })

# Display the corrected p-values
correction_results_df = pd.DataFrame(correction_results)

correction_results_df


,Classifier,Comparison,Bonferroni-corrected p-value,Rank-Biserial Correlation
0,LogisticRegression,Powerband vs CSP,0.012694,0.740000
1,LogisticRegression,Powerband vs Filterbank CSP,0.000013,0.973333
2,LogisticRegression,CSP vs Filterbank CSP,0.000549,0.873333
3,LDA,Powerband vs CSP,0.000245,0.900000
4,LDA,Powerband vs Filterbank CSP,0.000004,0.993333
5,LDA,CSP vs Filterbank CSP,0.000077,0.933333
6,SVM,Powerband vs CSP,0.000025,0.960000
7,SVM,Powerband vs Filterbank CSP,0.000004,0.993333
8,SVM,CSP vs Filterbank CSP,0.003589,0.800000
9,RandomForest,Powerband vs CSP,0.000013,0.973333


In [24]:
# Function to calculate rank-biserial correlation for Wilcoxon signed-rank test
def rank_biserial(w_stat, n):
    return 1 - (2 * w_stat / (n * (n + 1) / 2))

# Calculate effect sizes (rank-biserial correlation) for each pairwise Wilcoxon test
effect_sizes = []

for classifier, (test_type, results) in statistical_results.items():
    if isinstance(results, dict):  # Ensure there are pairwise comparisons
        for comparison, (w_stat, p_value) in results.items():
            # Number of observations (valid PIDs)
            n = len(valid_pids)

            # Calculate rank-biserial correlation
            rb_correlation = rank_biserial(w_stat, n)

            # Store the results
            effect_sizes.append({
                'Classifier': classifier,
                'Comparison': comparison,
                'Wilcoxon Stat': w_stat,
                'p-value': p_value,
                'Rank-Biserial Correlation': rb_correlation
            })

# Create a DataFrame for the effect sizes
effect_sizes_df = pd.DataFrame(effect_sizes)


effect_sizes_df


,Classifier,Comparison,Wilcoxon Stat,p-value,Rank-Biserial Correlation
0,LogisticRegression,Powerband vs CSP,39.0,8.462667e-04,0.740000
1,LogisticRegression,Powerband vs Filterbank CSP,4.0,8.344650e-07,0.973333
2,LogisticRegression,CSP vs Filterbank CSP,19.0,3.659725e-05,0.873333
3,LDA,Powerband vs CSP,15.0,1.633167e-05,0.900000
4,LDA,Powerband vs Filterbank CSP,1.0,2.384186e-07,0.993333
5,LDA,CSP vs Filterbank CSP,10.0,5.125999e-06,0.933333
6,SVM,Powerband vs CSP,6.0,1.668930e-06,0.960000
7,SVM,Powerband vs Filterbank CSP,1.0,2.384186e-07,0.993333
8,SVM,CSP vs Filterbank CSP,30.0,2.392530e-04,0.800000
9,RandomForest,Powerband vs CSP,4.0,8.344650e-07,0.973333


In [1]:
import pandas as pd
import numpy as np
from scipy.stats import shapiro, f_oneway, kruskal
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import scikit_posthocs as sp
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Load the data
classification1_data = pd.read_csv('classification_results.csv')
classification2_data = pd.read_csv('classification_results_2.csv')
classification3_data = pd.read_csv('classification_results_3_fbcsp.csv')

def perform_statistical_analysis(data, feature_extraction_name):
    print(f"\nStatistical Analysis for {feature_extraction_name} Feature Extraction Method")
    classifiers = data['classifier'].unique()
    accuracies = [data[data['classifier'] == clf]['mean_accuracy'] for clf in classifiers]
    all_accuracies = data['mean_accuracy']
    
    # Normality test for each classifier
    normality_results = {}
    print("\nNormality Test Results (Shapiro-Wilk):")
    for clf in classifiers:
        clf_data = data[data['classifier'] == clf]['mean_accuracy']
        stat, p = shapiro(clf_data)
        normality_results[clf] = p
        print(f"Classifier: {clf}, p-value: {p:.4f}")
    
    # Check if all classifiers have normally distributed data
    if all(p > 0.05 for p in normality_results.values()):
        print("\nData is normally distributed across all classifiers. Proceeding with One-way ANOVA.")
        # One-way ANOVA
        f_stat, p_value = f_oneway(*accuracies)
        print(f"ANOVA F-statistic: {f_stat:.4f}, p-value: {p_value:.4f}")
        
        # Effect Size (Eta Squared)
        grand_mean = np.mean(all_accuracies)
        ss_total = sum((all_accuracies - grand_mean) ** 2)
        ss_between = sum(len(acc) * (np.mean(acc) - grand_mean) ** 2 for acc in accuracies)
        eta_squared = ss_between / ss_total
        print(f"Effect Size (Eta Squared): {eta_squared:.4f}")
        
        # Post-hoc Tukey HSD test
        tukey = pairwise_tukeyhsd(endog=data['mean_accuracy'], groups=data['classifier'], alpha=0.05)
        print("\nPost-hoc Analysis (Tukey HSD):")
        print(tukey.summary())
    else:
        print("\nData is not normally distributed across all classifiers. Proceeding with Kruskal-Wallis Test.")
        # Kruskal-Wallis Test
        h_stat, p_value = kruskal(*accuracies)
        print(f"Kruskal-Wallis H-statistic: {h_stat:.4f}, p-value: {p_value:.4f}")
        
        # Effect Size (Epsilon Squared)
        n = len(data)
        k = len(classifiers)
        epsilon_squared = (h_stat - k + 1) / (n - k)
        print(f"Effect Size (Epsilon Squared): {epsilon_squared:.4f}")
        
        # Post-hoc Dunn's Test with Bonferroni correction
        dunn_results = sp.posthoc_dunn(data, val_col='mean_accuracy', group_col='classifier', p_adjust='bonferroni')
        print("\nPost-hoc Analysis (Dunn's Test with Bonferroni correction):")
        print(dunn_results)
    
    print("-" * 80)

# Perform statistical analysis on each feature extraction method
perform_statistical_analysis(classification1_data, 'Powerband')
perform_statistical_analysis(classification2_data, 'CSP')
perform_statistical_analysis(classification3_data, 'Filter Bank CSP')



Statistical Analysis for Powerband Feature Extraction Method

Normality Test Results (Shapiro-Wilk):
Classifier: LogisticRegression, p-value: 0.0077
Classifier: LDA, p-value: 0.0002
Classifier: SVM, p-value: 0.0810
Classifier: RandomForest, p-value: 0.0372
Classifier: KNN, p-value: 0.8508

Data is not normally distributed across all classifiers. Proceeding with Kruskal-Wallis Test.
Kruskal-Wallis H-statistic: 25.4897, p-value: 0.0000
Effect Size (Epsilon Squared): 0.1869

Post-hoc Analysis (Dunn's Test with Bonferroni correction):
                         KNN       LDA  LogisticRegression  RandomForest  \
KNN                 1.000000  0.022273            0.000023      0.374399   
LDA                 0.022273  1.000000            0.961289      1.000000   
LogisticRegression  0.000023  0.961289            1.000000      0.082636   
RandomForest        0.374399  1.000000            0.082636      1.000000   
SVM                 0.002895  1.000000            1.000000      1.000000   

     

In [28]:
import pandas as pd
import numpy as np
from scipy.stats import shapiro, f_oneway, friedmanchisquare
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import scikit_posthocs as sp
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Load the data
classification1_data = pd.read_csv('classification_results.csv')
classification2_data = pd.read_csv('classification_results_2.csv')
classification3_data = pd.read_csv('classification_results_3_fbcsp.csv')

def perform_statistical_analysis(data, feature_extraction_name):
    print(f"\nStatistical Analysis for {feature_extraction_name} Feature Extraction Method")
    classifiers = data['classifier'].unique()
    all_accuracies = data['mean_accuracy']
    
    # Ensure 'subject' column exists for repeated measures
    if 'subject' not in data.columns:
        # Assign a subject ID to each observation
        # Assumes that each classifier has the same number of observations
        n_subjects = int(len(data) / len(classifiers))
        data['subject'] = np.tile(range(1, n_subjects + 1), len(classifiers))
    
    # Normality test for each classifier
    normality_results = {}
    print("\nNormality Test Results (Shapiro-Wilk):")
    for clf in classifiers:
        clf_data = data[data['classifier'] == clf]['mean_accuracy']
        stat, p = shapiro(clf_data)
        normality_results[clf] = p
        print(f"Classifier: {clf}, p-value: {p:.4f}")
    
    # Prepare data for statistical tests
    accuracies = [data[data['classifier'] == clf]['mean_accuracy'].values for clf in classifiers]
    
    # Check if all classifiers have normally distributed data
    if all(p > 0.05 for p in normality_results.values()):
        print("\nData is normally distributed across all classifiers. Proceeding with One-way ANOVA.")
        # One-way ANOVA
        f_stat, p_value = f_oneway(*accuracies)
        print(f"ANOVA F-statistic: {f_stat:.4f}, p-value: {p_value:.4f}")
        
        # Effect Size (Eta Squared)
        grand_mean = np.mean(all_accuracies)
        ss_total = sum((all_accuracies - grand_mean) ** 2)
        ss_between = sum(len(acc) * (np.mean(acc) - grand_mean) ** 2 for acc in accuracies)
        eta_squared = ss_between / ss_total
        print(f"Effect Size (Eta Squared): {eta_squared:.4f}")
        
        # Post-hoc Tukey HSD test
        tukey = pairwise_tukeyhsd(endog=data['mean_accuracy'], groups=data['classifier'], alpha=0.05)
        print("\nPost-hoc Analysis (Tukey HSD):")
        print(tukey.summary())
    else:
        print("\nData is not normally distributed across all classifiers. Proceeding with Friedman Test.")
        # Friedman Test
        chi_square_stat, p_value = friedmanchisquare(*accuracies)
        print(f"Friedman Chi-Square statistic: {chi_square_stat:.4f}, p-value: {p_value:.4f}")
        
        # Effect Size (Kendall's W)
        n = len(data['subject'].unique())  # Number of subjects
        k = len(classifiers)  # Number of conditions
        kendalls_w = chi_square_stat / (n * (k - 1))
        print(f"Effect Size (Kendall's W): {kendalls_w:.4f}")
        
        # Post-hoc Nemenyi test
        nemenyi_results = sp.posthoc_nemenyi_friedman(data.groupby(['subject', 'classifier'])['mean_accuracy'].mean().reset_index(), y_col='mean_accuracy', block_col='subject', group_col='classifier', melted= True)
        print("\nPost-hoc Analysis (Nemenyi test):")
        print(nemenyi_results)
    
    print("-" * 80)

# Perform statistical analysis on each feature extraction method
perform_statistical_analysis(classification1_data, 'Powerband')
perform_statistical_analysis(classification2_data, 'CSP')
perform_statistical_analysis(classification3_data, 'Filter Bank CSP')



Statistical Analysis for Powerband Feature Extraction Method

Normality Test Results (Shapiro-Wilk):
Classifier: LogisticRegression, p-value: 0.0077
Classifier: LDA, p-value: 0.0002
Classifier: SVM, p-value: 0.0810
Classifier: RandomForest, p-value: 0.0372
Classifier: KNN, p-value: 0.8508

Data is not normally distributed across all classifiers. Proceeding with Friedman Test.
Friedman Chi-Square statistic: 59.8163, p-value: 0.0000
Effect Size (Kendall's W): 0.6231

Post-hoc Analysis (Nemenyi test):
                         KNN       LDA  LogisticRegression       SVM  \
KNN                 1.000000  0.006425            0.001000  0.016871   
LDA                 0.006425  1.000000            0.900000  0.900000   
LogisticRegression  0.001000  0.900000            1.000000  0.737028   
SVM                 0.016871  0.900000            0.737028  1.000000   
RandomForest        0.728394  0.182923            0.017703  0.316241   

                    RandomForest  
KNN                     0.7

In [27]:
# import pandas as pd
# import numpy as np
# from scipy.stats import shapiro, f_oneway, friedmanchisquare
# from statsmodels.stats.multicomp import pairwise_tukeyhsd
# import scikit_posthocs as sp
# import warnings

# # Suppress warnings for cleaner output
# warnings.filterwarnings('ignore')

# # Load the data
# classification1_data = pd.read_csv('classification_results.csv')
# classification2_data = pd.read_csv('classification_results_2.csv')
# classification3_data = pd.read_csv('classification_results_3_fbcsp.csv')

# def perform_statistical_analysis(data, feature_extraction_name):
#     print(f"\nStatistical Analysis for {feature_extraction_name} Feature Extraction Method")
#     classifiers = data['classifier'].unique()
#     all_accuracies = data['mean_accuracy']
    
#     # Ensure 'participant' column exists and is correctly assigned
#     if 'participant' not in data.columns:
#         n_classifiers = len(classifiers)
#         n_participants = int(len(data) / n_classifiers)
        
#         # Sort data by 'classifier' for consistent assignment
#         data = data.sort_values(by='classifier')
#         data['participant'] = np.repeat(range(1, n_participants + 1), n_classifiers)
        
#         # Re-sort data to original order if necessary
#         data = data.sort_index()
    
#     # Convert data types
#     data['mean_accuracy'] = pd.to_numeric(data['mean_accuracy'], errors='coerce')
#     data['participant'] = data['participant'].astype(str)
#     data['classifier'] = data['classifier'].astype(str)
#     data = data.dropna(subset=['mean_accuracy'])
    
#     # Normality test for each classifier
#     normality_results = {}
#     print("\nNormality Test Results (Shapiro-Wilk):")
#     for clf in classifiers:
#         clf_data = data[data['classifier'] == clf]['mean_accuracy']
#         stat, p = shapiro(clf_data)
#         normality_results[clf] = p
#         print(f"Classifier: {clf}, p-value: {p:.4f}")
    
#     # Prepare data for statistical tests
#     accuracies = [data[data['classifier'] == clf]['mean_accuracy'].values for clf in classifiers]
    
#     # Check if all classifiers have normally distributed data
#     if all(p > 0.05 for p in normality_results.values()):
#         print("\nData is normally distributed across all classifiers. Proceeding with One-way ANOVA.")
#         # One-way ANOVA
#         f_stat, p_value = f_oneway(*accuracies)
#         print(f"ANOVA F-statistic: {f_stat:.4f}, p-value: {p_value:.4f}")
        
#         # Effect Size (Eta Squared)
#         grand_mean = np.mean(all_accuracies)
#         ss_total = sum((all_accuracies - grand_mean) ** 2)
#         ss_between = sum(len(acc) * (np.mean(acc) - grand_mean) ** 2 for acc in accuracies)
#         eta_squared = ss_between / ss_total
#         print(f"Effect Size (Eta Squared): {eta_squared:.4f}")
        
#         # Post-hoc Tukey HSD test
#         tukey = pairwise_tukeyhsd(endog=data['mean_accuracy'], groups=data['classifier'], alpha=0.05)
#         print("\nPost-hoc Analysis (Tukey HSD):")
#         print(tukey.summary())
#     else:
#         print("\nData is not normally distributed across all classifiers. Proceeding with Friedman Test.")
#         # Friedman Test
#         chi_square_stat, p_value = friedmanchisquare(*accuracies)
#         print(f"Friedman Chi-Square statistic: {chi_square_stat:.4f}, p-value: {p_value:.4f}")
        
#         # Effect Size (Kendall's W)
#         n = len(data['participant'].unique())  # Number of participants
#         k = len(classifiers)  # Number of conditions
#         kendalls_w = chi_square_stat / (n * (k - 1))
#         print(f"Effect Size (Kendall's W): {kendalls_w:.4f}")
        
#         # Post-hoc Conover test with melted=True
#         conover_results = sp.posthoc_conover_friedman(
#             data,
#             y_col='mean_accuracy',
#             block_col='participant',
#             group_col='classifier',
#             melted=True,
#             p_adjust='bonferroni'
#         )
#         print("\nPost-hoc Analysis (Conover test with Bonferroni correction):")
#         print(conover_results)
    
#     print("-" * 80)

# # Perform statistical analysis on each feature extraction method
# perform_statistical_analysis(classification1_data, 'Powerband')
# perform_statistical_analysis(classification2_data, 'CSP')
# perform_statistical_analysis(classification3_data, 'Filter Bank CSP')



Statistical Analysis for Powerband Feature Extraction Method

Normality Test Results (Shapiro-Wilk):
Classifier: LogisticRegression, p-value: 0.0077
Classifier: LDA, p-value: 0.0002
Classifier: SVM, p-value: 0.0810
Classifier: RandomForest, p-value: 0.0372
Classifier: KNN, p-value: 0.8508

Data is not normally distributed across all classifiers. Proceeding with Friedman Test.
Friedman Chi-Square statistic: 59.8163, p-value: 0.0000
Effect Size (Kendall's W): 0.6231

Post-hoc Analysis (Conover test with Bonferroni correction):
                    LogisticRegression           LDA           SVM  \
LogisticRegression        1.000000e+00  2.890579e-06  1.242594e-01   
LDA                       2.890579e-06  1.000000e+00  3.609528e-02   
SVM                       1.242594e-01  3.609528e-02  1.000000e+00   
RandomForest              2.252788e-08  1.000000e+00  9.575673e-04   
KNN                       1.612421e-18  4.317433e-07  3.498565e-13   

                    RandomForest           KNN 

In [6]:
import pandas as pd
import numpy as np

# Load the data
classification1_data = pd.read_csv('classification_results.csv')
classification2_data = pd.read_csv('classification_results_2.csv')
classification3_data = pd.read_csv('classification_results_3_fbcsp.csv')

def calculate_mean_sd(data, feature_extraction_name):
    print(f"\nMean Accuracy and Standard Deviation for {feature_extraction_name} Feature Extraction Method")

    clf_data = data['mean_accuracy']
    mean_acc = clf_data.mean()
    std_acc = clf_data.std()
    stats = {'Mean Accuracy': mean_acc, 'Standard Deviation': std_acc}

    print(stats)

# Calculate mean and SD for each feature extraction method
calculate_mean_sd(classification1_data, 'Powerband')
calculate_mean_sd(classification2_data, 'CSP')
calculate_mean_sd(classification3_data, 'Filter Bank CSP')



Mean Accuracy and Standard Deviation for Powerband Feature Extraction Method
{'Mean Accuracy': np.float64(0.8162975969666667), 'Standard Deviation': np.float64(0.07222931694531354)}

Mean Accuracy and Standard Deviation for CSP Feature Extraction Method
{'Mean Accuracy': np.float64(0.9235203357249999), 'Standard Deviation': np.float64(0.05345525378441496)}

Mean Accuracy and Standard Deviation for Filter Bank CSP Feature Extraction Method
{'Mean Accuracy': np.float64(0.9425097556678886), 'Standard Deviation': np.float64(0.04507038871342122)}
